In [1]:
import torch.nn.functional as F
import pytorch_lightning as pl
import pickle
import numpy as np
import os
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import wandb

In [2]:
def process_radar_return(radar_return):
    # Ensure the radar return has shape (num_rows, num_cols)
    assert len(radar_return.shape) == 2

    # Define the target length of 512
    target_length = 512

    # Randomly select the starting index for the sequence
    start_index = np.random.randint(0, radar_return.shape[1] - 5)

    # Select the subsequent 4 indices to form a sequence of 5 adjacent pulses
    selected_pulses = np.arange(start_index, start_index + 6)

    # Concatenate selected pulses along rows
    concatenated_pulses = np.concatenate([radar_return[:, pulse] for pulse in selected_pulses], axis=0)

    # Take np.abs to convert complex numbers to real numbers
    epsilon = 1e-10
    processed_radar_return = concatenated_pulses # 10 * np.log10(((np.abs(concatenated_pulses))**2)+epsilon)

    # Ensure the processed radar return has length 512
    if processed_radar_return.shape[0] > target_length:
        # If length is greater than 512, truncate the vector
        processed_radar_return = processed_radar_return[:target_length]
    elif processed_radar_return.shape[0] < target_length:
        # If length is less than 512, pad with zeros
        print("error: default length is shorter than min length")
        # min_value = np.min(processed_radar_return)
        # pad_value = -200
        # processed_radar_return = np.pad(processed_radar_return,
        #                                 (0, target_length - processed_radar_return.shape[0]),
        #                                 mode='constant', constant_values=pad_value)

    return processed_radar_return

In [3]:
def normalize_radar_return_column(combined_df):
    radar_data = combined_df['radar_return']

    # Flatten the radar data for scaler fitting
    flattened_data = [np.array(row).flatten() for row in radar_data]
    flattened_data = np.concatenate(flattened_data).reshape(-1, 1)

    # Fit the scaler on the flattened data
    scaler = MinMaxScaler()
    scaler.fit(flattened_data)

    # Transform each row separately
    normalized_data = [scaler.transform(np.array(row).reshape(-1, 1)).flatten().tolist() for row in radar_data]

    # Reshape the normalized data back to its original shape
    combined_df['radar_return'] = normalized_data

    return combined_df

In [4]:
data_dir = 'First_data'  # Change this to your data folder path
dataframes = []

# set device!
print(torch.cuda.is_available())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

True


In [5]:
print("creating the dataframe...")
for filename in os.listdir(data_dir):
    if filename.endswith('.pickle'):
        file_path = os.path.join(data_dir, filename)
        with open(file_path, 'rb') as f:
            # Load data from pickle file
            data = pickle.load(f)
            # Extract radar_return and object_id
            radar_return = data['radar_return']
            object_id = data['object_id']
            # Concatenate radar_return along the columns
            concatenated_radar = process_radar_return(radar_return).astype('float32')
            # Create a DataFrame with concatenated radar and object_id
            df = pd.DataFrame({'radar_return': [concatenated_radar], 'object_id': [object_id]})
            # Append the DataFrame to the list
            dataframes.append(df)

# Concatenate all DataFrames into one
combined_df = pd.concat(dataframes[::50], ignore_index=True)
#combined_df = normalize_radar_return_column(combined_df)
print(normalize_radar_return_column(combined_df))

creating the dataframe...
                                          radar_return  \
0    [0.0015318399528041482, 0.00626467214897275, 0...   
1    [7.205681322375312e-05, 0.001935760723426938, ...   
2    [0.0029054523911327124, 0.006725980434566736, ...   
3    [0.0032423753291368484, 0.0037096706219017506,...   
4    [0.00011989560152869672, 8.637927385279909e-05...   
..                                                 ...   
395  [0.00011174243991263211, 6.571057747351006e-05...   
396  [0.0006189570412971079, 0.0016693189973011613,...   
397  [0.00016235027578659356, 0.0002097746328217908...   
398  [0.00342611875385046, 0.01031186431646347, 0.0...   
399  [0.01176843885332346, 0.023287517949938774, 0....   

                            object_id  
0    1c47c37d6c785d4fe7fd9a18c19837e5  
1    1c47c37d6c785d4fe7fd9a18c19837e5  
2    1c47c37d6c785d4fe7fd9a18c19837e5  
3    1c47c37d6c785d4fe7fd9a18c19837e5  
4    1c47c37d6c785d4fe7fd9a18c19837e5  
..                                ...

In [6]:
class FNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(FNN, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.BatchNorm1d(hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.BatchNorm1d(hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.BatchNorm1d(hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.BatchNorm1d(hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.BatchNorm1d(hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, output_size)
        )

    def forward(self, x):
        return self.model(x)

In [7]:

class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=3):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.tanh = nn.Tanh()
        self.softmax = nn.LogSoftmax(1)

    def forward(self, x):
        x = x.unsqueeze(-1)  # Add a new dimension at the end
        out, _ = self.lstm(x)
        out = out[: ,-1]

        out = self.tanh(out)

        out = self.fc(out)
        final = self.softmax(out)# Take the last time step's output
        return final


In [8]:
class RadarDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]


In [9]:
radar_data = torch.tensor(combined_df['radar_return'].values.tolist(), dtype=torch.float32)
object_ids = combined_df['object_id'].values.tolist()
# Use LabelEncoder to convert object_ids to numerical labels
label_encoder = LabelEncoder()
object_ids_encoded = label_encoder.fit_transform(object_ids)
# print(radar_data[0])
# print(object_ids_encoded)

# Convert encoded labels to tensor
object_ids_tensor = torch.tensor(object_ids_encoded, dtype=torch.float32)
data_train, data_test, label_train, label_test = train_test_split(radar_data, object_ids_encoded, stratify=object_ids_encoded)

# Define hyperparameters
input_size = 1 #radar_data.shape[1]
hidden_size = 512
output_size = 10
learning_rate = 0.001
num_epochs = 500
batch_size = 32

# Create FNN instance
model = LSTM(input_size, hidden_size, output_size)

# Define loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Create DataLoader for batch training
dataset = RadarDataset(data_train, label_train)
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

val_dataset = RadarDataset(data_test, label_test)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

wandb.init(project='LSTM_notebook', entity='mfbaysan')

wandb: Currently logged in as: mfbaysan. Use `wandb login --relogin` to force relogin


In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move model to GPU if available
model.to(device)

LSTM(
  (lstm): LSTM(1, 512, num_layers=3, batch_first=True)
  (fc): Linear(in_features=512, out_features=10, bias=True)
  (tanh): Tanh()
  (softmax): LogSoftmax(dim=1)
)

In [11]:

# Training loop
for epoch in range(num_epochs):
    # Training phase
    model.train()
    train_correct_predictions = 0
    train_total_samples = 0
    train_total_loss = 0.0

    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)
        loss = criterion(outputs, labels.view(-1).long())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        _, predicted = torch.max(outputs, 1)
        train_correct_predictions += (predicted == labels).sum().item()
        train_total_samples += labels.size(0)

        train_total_loss += loss.item()

        avg_train_loss = train_total_loss / (i + 1)
        train_accuracy = train_correct_predictions / train_total_samples

        print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Train Loss: {avg_train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}')

    # Validation phase
    model.eval()
    val_correct_predictions = 0
    val_total_samples = 0
    val_total_loss = 0.0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels.view(-1).long())

            _, predicted = torch.max(outputs, 1)
            val_correct_predictions += (predicted == labels).sum().item()
            val_total_samples += labels.size(0)

            val_total_loss += loss.item()

    avg_val_loss = val_total_loss / len(val_loader)
    val_accuracy = val_correct_predictions / val_total_samples

    print(f'Epoch [{epoch+1}/{num_epochs}], Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}')

    wandb.log({
        "Train Loss": avg_train_loss,
        "Train Accuracy": train_accuracy,
        "Validation Loss": avg_val_loss,
        "Validation Accuracy": val_accuracy
    }, step=epoch)

print('Training finished.')

Epoch [1/500], Step [1/10], Train Loss: 2.3021, Train Accuracy: 0.0312
Epoch [1/500], Step [2/10], Train Loss: 2.3127, Train Accuracy: 0.0469
Epoch [1/500], Step [3/10], Train Loss: 2.3157, Train Accuracy: 0.0521
Epoch [1/500], Step [4/10], Train Loss: 2.3102, Train Accuracy: 0.0859
Epoch [1/500], Step [5/10], Train Loss: 2.3119, Train Accuracy: 0.0750
Epoch [1/500], Step [6/10], Train Loss: 2.3100, Train Accuracy: 0.0833
Epoch [1/500], Step [7/10], Train Loss: 2.3117, Train Accuracy: 0.0759
Epoch [1/500], Step [8/10], Train Loss: 2.3119, Train Accuracy: 0.0781
Epoch [1/500], Step [9/10], Train Loss: 2.3115, Train Accuracy: 0.0799
Epoch [1/500], Step [10/10], Train Loss: 2.3111, Train Accuracy: 0.0800
Epoch [1/500], Validation Loss: 2.3013, Validation Accuracy: 0.1000
Epoch [2/500], Step [1/10], Train Loss: 2.2957, Train Accuracy: 0.0625
Epoch [2/500], Step [2/10], Train Loss: 2.3057, Train Accuracy: 0.0625
Epoch [2/500], Step [3/10], Train Loss: 2.3082, Train Accuracy: 0.0521
Epoch [2